In [1]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import classification_report
import keras
import numpy as np
import csv
import random

Using TensorFlow backend.


In [2]:
infile = open('../input/voiceidentificationnew/features_new.csv')
tests = open('../input/voiceidentificationnew/test_indices_new.csv')
trains = open('../input/voiceidentificationnew/train_indices_new.csv')
csvreader = csv.reader(infile)
csv1 = csv.reader(tests)
csv2 = csv.reader(trains)

data = list(csvreader)
training_data = [data[int(i[0])] for i in csv2]
testing_data = [data[int(i[0])] for i in csv1]
training_labels = np.array([[int(i[0])] for i in training_data])
testing_labels = np.array([[int(i[0])] for i in testing_data])
training_data = np.array([i[1:] for i in training_data])
testing_data = np.array([i[1:] for i in testing_data])

Y = np.concatenate((training_labels,testing_labels),axis=0)
X = np.concatenate((training_data,testing_data),axis=0)

infile.close()
tests.close()
trains.close()

In [3]:
model = Sequential([Dense(1024, activation='relu', input_shape=(len(training_data[0]),)), Dropout(0.5), Dense(256, activation='relu'), Dropout(0.5), Dense(64, activation='relu'), Dropout(0.5), Dense(10, activation='softmax')])
model.load_weights("../input/weightsafter91epochs/weights-improvement-91-0.63.hdf5")
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [4]:
filepath="weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

one_hot_labels = keras.utils.to_categorical(Y, num_classes=10)
model.fit(X, one_hot_labels, validation_split=0.1, epochs=100, batch_size=32, callbacks=callbacks_list, verbose=1)


Train on 29809 samples, validate on 3313 samples
Epoch 1/100
29809/29809 [==============================] - 8s 253us/step - loss: 0.9921 - accuracy: 0.6077 - val_loss: 1.0873 - val_accuracy: 0.6206

Epoch 00001: val_accuracy improved from -inf to 0.62059, saving model to weights-improvement-01-0.62.hdf5
Epoch 2/100
29809/29809 [==============================] - 7s 238us/step - loss: 0.9877 - accuracy: 0.6133 - val_loss: 1.0677 - val_accuracy: 0.6167

Epoch 00002: val_accuracy did not improve from 0.62059
Epoch 3/100
29809/29809 [==============================] - 7s 239us/step - loss: 0.9869 - accuracy: 0.6114 - val_loss: 1.0561 - val_accuracy: 0.6085

Epoch 00003: val_accuracy did not improve from 0.62059
Epoch 4/100
29809/29809 [==============================] - 7s 247us/step - loss: 0.9847 - accuracy: 0.6128 - val_loss: 1.1019 - val_accuracy: 0.6161

Epoch 00004: val_accuracy did not improve from 0.62059
Epoch 5/100
29809/29809 [==============================] - 7s 246us/step - loss:

In [5]:
one_hot_test_labels = keras.utils.to_categorical(testing_labels, num_classes=10)
score = model.evaluate(testing_data, one_hot_test_labels)
print(score)

4000/4000 [==============================] - 0s 94us/step
[1.3138993587493897, 0.624750018119812]


In [6]:
y_pred = model.predict(testing_data)
report = classification_report(testing_labels, [np.argmax(i) for i in y_pred])

In [7]:
print(report)

              precision    recall  f1-score   support

           0       0.61      0.60      0.61       791
           1       0.63      0.68      0.66       840
           2       0.59      0.79      0.68       826
           3       0.76      0.57      0.65       792
           4       0.57      0.47      0.51       751

    accuracy                           0.62      4000
   macro avg       0.63      0.62      0.62      4000
weighted avg       0.63      0.62      0.62      4000

